In [44]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

import warnings
warnings.filterwarnings('ignore')

In [45]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
df = pd.read_csv("Resources/credit_risk_dataset.csv")
# Review the DataFrame
df.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4


In [46]:
df.columns

Index(['person_age', 'person_income', 'person_home_ownership',
       'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt',
       'loan_int_rate', 'loan_status', 'loan_percent_income',
       'cb_person_default_on_file', 'cb_person_cred_hist_length'],
      dtype='object')

In [47]:
consolidated_df = df.drop(columns=[
       'person_emp_length', 'loan_intent', 'loan_grade',
       'loan_int_rate', 'loan_percent_income',
       'cb_person_default_on_file', 'cb_person_cred_hist_length'])

In [48]:
# Create a list of categorical variables 
categorical_variables = list(consolidated_df.dtypes[consolidated_df.dtypes=="object"].index)

# Display the categorical variables list
categorical_variables

['person_home_ownership']

In [49]:
from sklearn.preprocessing import StandardScaler,OneHotEncoder

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

In [50]:
encoded_data = enc.fit_transform(consolidated_df[categorical_variables])

In [51]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(
    encoded_data,
columns=enc.get_feature_names_out(categorical_variables)
)

# Review the DataFrame
encoded_df.head()

,person_home_ownership_MORTGAGE,person_home_ownership_OTHER,person_home_ownership_OWN,person_home_ownership_RENT
0,0.0,0.0,0.0,1.0
1,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0


In [52]:
# create a new df that does not include those columns we converted with one_hot_encoder (all numerical variables from original dataset)

numerical_variables_df = consolidated_df.drop(columns=categorical_variables)

#review the dataframe
numerical_variables_df.head()

,person_age,person_income,loan_amnt,loan_status
0,22,59000,35000,1
1,21,9600,1000,0
2,25,9600,5500,1
3,23,65500,35000,1
4,24,54400,35000,1


In [53]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame


encoded_df = pd.concat([encoded_df, numerical_variables_df], axis=1)

# Review the Dataframe
encoded_df.head()

,person_home_ownership_MORTGAGE,person_home_ownership_OTHER,person_home_ownership_OWN,person_home_ownership_RENT,person_age,person_income,loan_amnt,loan_status
0,0.0,0.0,0.0,1.0,22,59000,35000,1
1,0.0,0.0,1.0,0.0,21,9600,1000,0
2,1.0,0.0,0.0,0.0,25,9600,5500,1
3,0.0,0.0,0.0,1.0,23,65500,35000,1
4,0.0,0.0,0.0,1.0,24,54400,35000,1


In [54]:
# separate the data into features and target

y = encoded_df["loan_status"]

In [55]:
y[:7]

0    1
1    0
2    1
3    1
4    1
5    1
6    1
Name: loan_status, dtype: int64

In [56]:
X = encoded_df.drop(columns="loan_status")

In [57]:
X.head()

,person_home_ownership_MORTGAGE,person_home_ownership_OTHER,person_home_ownership_OWN,person_home_ownership_RENT,person_age,person_income,loan_amnt
0,0.0,0.0,0.0,1.0,22,59000,35000
1,0.0,0.0,1.0,0.0,21,9600,1000
2,1.0,0.0,0.0,0.0,25,9600,5500
3,0.0,0.0,0.0,1.0,23,65500,35000
4,0.0,0.0,0.0,1.0,24,54400,35000


In [58]:
# check the value_counts to see whether there is an oversampling issue
y.value_counts()

0    25473
1     7108
Name: loan_status, dtype: int64

In [59]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [60]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
random_oversampler = RandomOverSampler(random_state=1)
# Fit the training data to the random_oversampler model
X_resampled, y_resampled = random_oversampler.fit_resample(X_train, y_train)

In [61]:
y_resampled.value_counts()

0    19113
1    19113
Name: loan_status, dtype: int64

In [62]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [63]:
from sklearn.linear_model import LogisticRegression


# Instantiate the Logistic Regression model
logistic_regression_model = LogisticRegression(random_state=1)
# Fit the model using the resampled training data
lr_model = logistic_regression_model.fit(X_resampled, y_resampled)
# Make a prediction using the testing data
predictions_oversampled = logistic_regression_model.predict(X_test)

In [64]:
balanced_accuracy_score(y_test, predictions_oversampled)

0.6767254220456802

In [65]:
confusion_matrix(y_test, predictions_oversampled)

array([[4089, 2271],
       [ 517, 1269]], dtype=int64)

In [66]:
print(classification_report_imbalanced(y_test, predictions_oversampled))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.89      0.64      0.71      0.75      0.68      0.45      6360
          1       0.36      0.71      0.64      0.48      0.68      0.46      1786

avg / total       0.77      0.66      0.70      0.69      0.68      0.46      8146

